In [3]:
import pandas as pd
import pandas as np
import time

import re
import requests
from bs4 import BeautifulSoup

from scipy.spatial.distance import cosine

import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer

In [4]:
def news_crwaling(search):
    headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}

    # 검색어 설정
    search_query = search

    # 검색 결과 페이지의 URL입니다.
    base_url = 'https://search.naver.com/search.naver'
    url = f"https://search.naver.com/search.naver?query={search_query}"

    # HTTP GET 요청을 보내고 응답을 받습니다.
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 뉴스 메뉴의 탭 요소를 가져옵니다.
    news_menu = soup.find("div", {"class": "lnb_group"}).find("ul").find_all("a")

    # 뉴스 메뉴의 탭의 링크 URL을 불러온다.
    for menu in news_menu:
        if menu.text=='뉴스':
            url = base_url + menu.get('href')
            break
    
    # parsing
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    news_list = soup.find_all('a', 'news_tit')
    news_info = soup.find_all('div', 'news_info')

    news_title = []
    news_url = []
    news_if = []

    for news_box in news_list:
        news_title.append(news_box.get_text())
    for url in news_list:
        news_url.append(url['href'])
    for info in news_info:
        news_if.append(info.get_text())

    news_content = []

    for url in news_url:
        # 본문 내용 가져오기
        try:
            response = requests.get(url, headers=headers)
        except:
            pass
        soup = BeautifulSoup(response.text, 'html.parser')

        # 본문 내용 추출
        content = ''
        for p in soup.find_all('p'):
            content += p.get_text()
        news_content.append(content)
    
    # 본문 전처리 >> 기사마다 HTML 구성이 다 다르다. 하나하나 다 전처리하기는 사실상 불가능하다.
    # 따라서 전처리가 잘 되지 않은 기사들은 cossim_model에서 걸러내기로 하였다.
    news_cnt_sr = []

    for news in news_content:
        d = re.sub(r'(\d{3})-(\d{4})-(\d{4})', ' ', news)
        d = re.sub(r'(\d{4})-(\d{2})-(\d{2}) (\d{2}):(\d{2}):(\d{2})', ' ', d)
        d = re.sub(r'(\d{4})-(\d{2})-(\d{2}) (\d{2}):(\d{2})', ' ', d)
        d = re.sub(r'(\d{4})-(\d{2})-(\d{2})', ' ', d)
        d = re.sub(r'(\d{4}).(\d{2}).(\d{2})', ' ', d)
        d = re.sub(r'(\d{4}):(\d{2}):(\d{2})', ' ', d)
        d = re.sub(r'(\d{4})-(\d{4})', ' ', d)
        d = re.sub(r'[\n\t\r]', '', d)
        d = d.split('@')[0]
        news_cnt_sr.append(d)

    # 뉴스정보 전처리
    news_if_sr = []
    
    for info in news_if:
        info = ''.join(str(info).split()[-3:]).split()
        info = info[0][:-5].split('선정')
        news_if_sr.append(info)
    news_if_sr    

    return news_title, news_if_sr, news_cnt_sr

In [5]:
def sum_model(news_cnt_sr):

    tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
    model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

    news_cnt_sr = news_cnt_sr
    news_sum = []
    summary = ''

    for news_cnt in news_cnt_sr:
        text = news_cnt

        text = text.replace('\n', ' ')

        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

        try:
            summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
            summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
            if len(summary) < 200: # 너무 긴 summary는 압축되지 못하는 parsing이 잘못되는 경우가 대부분이다.
                news_sum.append(summary)
            else:
                news_sum.append('')
        except:
            news_sum.append('')
    
    
    return news_sum

In [6]:
def cossim_model(news_title, news_sum):
    news_title = news_title
    news_sum = news_sum
    cos_sim = []

    model = SentenceTransformer('paraphrase-distilroberta-base-v1')
    # 문장 입력
    for i in range(len(news_title)):
        sentence1 = news_title[i]
        sentence2 = news_sum[i]

        # 두 문장을 벡터로 변환
        embeddings1 = model.encode(sentence1, convert_to_tensor=True)
        embeddings2 = model.encode(sentence2, convert_to_tensor=True)

        # 두 벡터 간 코사인 유사도를 계산
        cosine_similarity = 1 - cosine(embeddings1, embeddings2)

        # 유사도가 0.5 이상만 추출 >> 본문과 요약문이 동떨어진 기사 제거
        if cosine_similarity >= 0.5:
            cos_sim.append(cosine_similarity)
        else: cos_sim.append('')
    return cos_sim

In [7]:
def main(search):
    search = search
    news_title, news_if_sr, news_cnt_sr =  news_crwaling(search)
    news_sum = sum_model(news_cnt_sr)
    cos_sim = cossim_model(news_title, news_sum)
    
    df = []
    df = pd.DataFrame({'Title':news_title, 'Content':news_sum, 'Similiarity':cos_sim})
    df = df[~(df == '').any(axis=1)]
    # 요약시 발생하는 dummy 값이다. 따라서 제거해준다.
    df = df[~(df == '한국토지자원관리공단은 한국토지공사의 지분참여를 통해 일자리 창출을 도모할 예정이다.').any(axis=1)].reset_index(drop=True)

    return news_title, news_if_sr, news_cnt_sr, news_sum, cos_sim, df

In [8]:
if __name__ == '__main__':
    search = '삼성전자'
    news_title, news_if_sr, news_cnt_sr, news_sum, cos_sim, df = main(search)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [9]:
df

,Title,Content,Similiarity
0,"삼성전자, 자율주행車 반도체 생산…TSMC 추격 '고삐'",삼성전전자가 첨단 5나노미터 파운드리(반도체 위탁생산) 공정으로 자율주행 차량용 반...,0.557315
1,"삼성전자, 17년 연속 글로벌 TV 시장 판매 1위 달성",삼성전자[005930]가 글로벌 경기 침체에 따른 TV 수요 둔화에도 세계 TV 시...,0.934377
2,유진투자증권 '예드자문형 랩 삼성전자' 출시,"‘íê·1ê ̧°+ì°°C, 인테 인테 인테 인테리어로 이루어진 ‘íê·...",0.616622
3,삼성전자 2023년형 TV 내달 9일 출격…OLED 국내 첫선,삼성전자는 202 TV 신제품에 인공지능(AI) 기술을 적용해 최상의 화질을 제공하...,0.672162


In [25]:
dictionary = {i : string for i,string in enumerate(df.Content.values)}

dictionary

{0: "삼성전자는 202 TV 신제품에 인공지능(AI) 기술을 적용해 최상의 화질을 제공하고 다양한 기기와의 연결 경험을 극대화한 '네오 QLED 8K' 등 2023년형 TV 신제품을 다음 달 9일 출시한다.",
 1: '삼성전자가 잠정 4분기 매출 70조원, 영업이익 4조3000억원을 달성했다고 밝혔는데, 매출은 전년 대비 8.6% 줄었고, 영업이익은 전년 대비 69% 감소했다.',
 2: "삼성전자[005930]는 3월 9일 공식 출시에 앞서 QLED 8K, 네오 QLED, OLED 모델을 먼저 판매하고 3월 1일부터 삼성 삼성 디지털프라자와 백화점에서 사전 구매 고객에게는 휴대용 프로젝터 '더 프리스타일', JBL 게이밍 헤드셋, 티빙 프리미엄 이용권 등 다양한 혜택을 제공한다.",
 3: '10:54집닥이 삼성전자와 함께 집닥 인테리어 고객에게 TV, 세탁기, 인덕션, 에어컨 등 삼성전자 가전제품 인기 상품을 최대 43% 할인된 가격으로 제공하는 제휴 이벤트를 진행한다.',
 4: '20일 LG전전자는 유럽경제위원회 회원국에 차량을 판매하는 완성차 고객에게 검증된 사이버보안을 갖춘 인포테인먼트 시스템과 텔레매틱스 부품을 안정적으로 공급하게 되면서 글로벌 전장 시장 공략에 힘이 더 실릴 전망이다.'}

In [28]:
if __name__ == '__main__':
    search = 'SK하이닉스'
    news_title, news_if_sr, news_cnt_sr, news_sum, cos_sim, df = main(search)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [29]:
df

,Title,Content,Similiarity
0,"박정호 SK하이닉스 부회장 ""AI 챗봇 덕 반도체 수요↑""",박정호 SK하이닉스 부회장은 15일 서울 광화문 프레스센터에서 열린 한림대 도헌학술...,0.517653
1,범상찮은 '챗GPT' 신드롬…삼성전자‧SK하이닉스 주가 기지개 켜나,전 세계적으로 인공지능(AI) 챗봇 서비스 '챗GPT'가 출시 2개월 만인 올해 1...,0.577231
2,"박정호 SK하이닉스 부회장 ""IT 진화는 결국 메모리…챗GPT도 그래""",보험업계 보험사 관계자는 보험 가입자가 보험금 수령시 보험금 수령금액에 따라 보험금...,0.697403


In [30]:
df.Content.values

array(['박정호 SK하이닉스 부회장은 15일 서울 광화문 프레스센터에서 열린 한림대 도헌학술원 개원 기념 학술 발표회에서 "인공지능 대화 로봇(AI Chatbot)으로 반도체 수요가 폭발적으로 늘 것"이라며 "챗GPT를 비롯한 AI 기술이 진화하면서 세계적으로 데이터 생성·저장·처리량이 기하급수적으로 늘어날 것"이라고 내다봤다.',
       "전 세계적으로 인공지능(AI) 챗봇 서비스 '챗GPT'가 출시 2개월 만인 올해 1월 월간활성사용자(MAU) 1억 명을 넘어섰고, 삼성전자와 SK하이닉스의 주가 행보도 주목을 받고 있다.",
       '보험업계 보험사 관계자는 보험 가입자가 보험금 수령시 보험금 수령금액에 따라 보험금을 지급하고 보험료를 돌려받는다고 전했다.'],
      dtype=object)

In [40]:
if __name__ == '__main__':
    search = '하이브'
    news_title, news_if_sr, news_sum, news_cnt_sr, cos_sim, df = main(search)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [41]:
df

,Title,Content,Similiarity
0,"하이브 CEO ""SM 역외탈세, 결코 일어날 수 없는 시나리오""",하이브 박지원 CEO는 17일 오전 전 직원에게 보낸 사내 이메일을 통해 SM 이성...,0.837687
1,"SM ""하이브, 이수만 역외탈세 의혹 동조냐 묵인이냐"" 맹공",대한가수협회 이자연 회장은 16일 오후 협회 임원들과 긴급 회동을 한 후 SM 사태...,0.648157
2,이틀째 이어진 하이브-SM 공방... '역외탈세 의혹' 해소될까,하이브와 이 전 총괄 프로듀서 간 계약을 적대적 인수합병(M&A)으로 규정하며 반발...,0.840749
3,하이브家만 11억..뉴진스·BTS→방시혁 기부 행렬[★FOCUS],스타뉴스 연예 2팀 이승훈 기자입니다.,0.522155
4,몸값 치솟은 SM…하이브-카카오 '쩐의전쟁' 반격 또 반격,보험업계 보험사 관계자는 보험 가입자가 보험금 수령시 보험금 수령금액에 따라 보험금...,0.628034


In [42]:
df.Content.values

array(['하이브 박지원 CEO는 17일 오전 전 직원에게 보낸 사내 이메일을 통해 SM 이성수 현 대표이사가 제기한 이수만 전 총괄 프로듀서의 역외탈세 의혹 등에 대해 "결코 일어날 수 없는 시나리오"라고 거듭 우려 불식에 나섰다.',
       '대한가수협회 이자연 회장은 16일 오후 협회 임원들과 긴급 회동을 한 후 SM 사태와 관련 안타까움과 우려를 표하며 SM의 구조적인 문제와 갈등을 넘어서서 그가 불명예 퇴진을 한다는 것은 가요계의 비극이 될 것이라고 말했다.',
       '하이브와 이 전 총괄 프로듀서 간 계약을 적대적 인수합병(M&A)으로 규정하며 반발하는 SM 간 진흙탕 싸움이 이틀째 이어졌다.',
       '스타뉴스 연예 2팀 이승훈 기자입니다.',
       '보험업계 보험사 관계자는 보험 가입자가 보험금 수령시 보험금 수령금액에 따라 보험금을 지급하고 보험료를 돌려받는다고 전했다.'],
      dtype=object)

In [34]:
if __name__ == '__main__':
    search = '마이크로소프트'
    news_title, news_if_sr, news_cnt_sr, news_sum, cos_sim, df = main(search)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [35]:
df

,Title,Content,Similiarity
0,"마이크로소프트, 오류 논란 속 빙 검색 AI 챗봇 수정 검토..""제한 조치도 추가...",마이크로소프트소프트가 오픈AI 챗봇AI 기술을 기반으로 빙 검색엔진에 탑재한 챗봇A...,0.886328
1,"[글로벌] 마이크로소프트, 산업용 메타버스 부문을 폐쇄…그 의도는?",마이크로소프트가 2022년 10월 출범한 산업용 메타버스 프로젝트를 포기하고 관련 ...,0.682705
2,마이크로소프트(MS) 주가 2.66% 하락 262.15 달러 기록,마이크로소프트소프트(MS) 주가가 생성형 인공지능(AI)을 장착한 구글의 새 검색 ...,0.647473
3,"챗GPT 너무 의식했나? 구글-마이크로소프트의 AI 챗봇 빙(Bing), 바드(Ba...",15일 CNN 등 매체에 따르면 MS는 지난 7일 미국 워싱턴주 레드먼드 본사에서 ...,0.509265


In [36]:
df.Content.values

array(['마이크로소프트소프트가 오픈AI 챗봇AI 기술을 기반으로 빙 검색엔진에 탑재한 챗봇AI가 놀랍고 이상하게 반응하는 것을 걸러 내기 위해 사용지들에게 대화를 다시 시작하고 어조(tone)에 대해 보다 많이 통제할 수 있는 도구를 추가하는 것을 검토하고 있다.',
       '마이크로소프트가 2022년 10월 출범한 산업용 메타버스 프로젝트를 포기하고 관련 사업부를 닫은 것으로 알려졌는데, 마이크로소프트는 이 프로젝트에서 증강현실 헤드셋 홀로렌즈2(HoloLens2)를 이용해 발전소나 산업용 로봇, 수송 네트워크의 제어 시스템 등을 운용하기 위한 소프트웨어 인터페이스를 개발하고 있었다.',
       "마이크로소프트소프트(MS) 주가가 생성형 인공지능(AI)을 장착한 구글의 새 검색 엔진 '바드'뿐만 아니라 마이크로소프트(MS)의 검색 엔진 '빙'도 오답을 제시한 것으로 알려짐에 따라 전거래일보다 2.66% 하락한 262.15 달러를 기록했다.",
       '15일 CNN 등 매체에 따르면 MS는 지난 7일 미국 워싱턴주 레드먼드 본사에서 챗GPT와 같은 생성형 AI를 탑재한 검색 엔진 ‘빙’의 출시를 발표하며 당시 초청된 현지 미디어에 빙의 기능을 시연했는데, 빙은 매출과 순이익 등의 실적을 요약해 제시했으며, 이어 갭의 총마진율은 37.4%, 영업마진율은 4.6%로 각각 제시했다.'],
      dtype=object)

In [37]:
if __name__ == '__main__':
    search = '라오어 드라마'
    news_title, news_if_sr, news_cnt_sr, news_sum, cos_sim, df = main(search)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [38]:
df

,Title,Content,Similiarity
0,"'페이데이' 시리즈 실사화 진행된다, '라오어'처럼 성공할까?",아이디비밀번호범죄 액션 게임 <페이데이> 시리즈가 프로덕션 회사 스톡홀름 신드롬과 ...,0.829519
1,"너티독, '라오어' PC 버전 출시 연기 ... 최적화 담금질",너티독이 3인칭 액션 어드벤처 게임인 '더 라스트 오브 어스(이하 라오어)' PC ...,0.668275
2,마리오부터 라오어까지…게임 IP 영화·드라마 대거 공개,"슈퍼마리오, 그란 투리스모, 더 라스트 오브 어스(라스트오브어스) 등 유명 게임 I...",0.939439
3,‘역대급’ 일반 시청자 호평 쏟아지는 HBO ‘라오어’ 드라마,15일(현지시간) 미국 HBO 방송을 통해 공개된 <더 라스트 오브 어스> 1화에 ...,0.679541
4,"'라오어' 드라마, 방영 이틀 만에 1천만 명 시청 '대박'","HBO는 ‘라오어’ 드라마가 방영 후 2일 만에 누적 시청자 수 1,000만 명을 ...",0.611998


In [39]:
df.Content.values

array(['아이디비밀번호범죄 액션 게임 <페이데이> 시리즈가 프로덕션 회사 스톡홀름 신드롬과 파트너십을 맺어 영화나 TV 드라마로 실사화를 준비하고 있다.',
       "너티독이 3인칭 액션 어드벤처 게임인 '더 라스트 오브 어스(이하 라오어)' PC 버전의 출시를 3월 28일(한국시간 3월 29일)로 연기했다.",
       '슈퍼마리오, 그란 투리스모, 더 라스트 오브 어스(라스트오브어스) 등 유명 게임 IP 기반 영상물이 올해 대거 공개된다.',
       '15일(현지시간) 미국 HBO 방송을 통해 공개된 <더 라스트 오브 어스> 1화에 대한 호평이 쏟아지고 있다.',
       'HBO는 ‘라오어’ 드라마가 방영 후 2일 만에 누적 시청자 수 1,000만 명을 돌파했다고 발표했는데, ‘라오어’ 드라마는 첫 번째 에피소드 방영 직후 470만 명의 시청자 수를 기록했고, 이는 ‘왕좌의 게임’ 스핀오프 작인 ‘하우스 오브 더 드래곤’에 이어 HBO 드라마 역사상 두 번째로 높은 수치이다.'],
      dtype=object)

# practice

In [16]:
if __name__ == '__main__':
    search = '뉴진스'
    news_title, news_if_sr, news_cnt_sr, news_sum, cos_sim, df = main(search)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [17]:
df

,Title,Content,Similiarity
0,"뉴진스, 데뷔음반도 밀리언셀러…'OMG' 이어 두 번째",뉴진스는 데뷔음반도 밀리언셀러...'OMG' 이어 두 번째 등록을 마쳤으며 08:1...,0.935069
1,"하이브 작년 매출 1.7조 역대 최대…""지민 3월 솔로 앨범""(종합2보)",그룹 방탄소년단(BTS)의 소속사 하이브가 글로벌 팬덤 확장과 신인의 성공적인 데뷔...,0.659652
2,"뉴진스, 미국 누적 앨범 판매량 10만장 돌파..민희진도 '축하'",스타뉴스 가요 담당 윤상근 기자입니다.,0.769488
3,"""그게 사실은…"" 뉴진스 민지, 레전드 '비눗방울' 직캠 비하인드 공개",그룹 뉴진스 멤버 민지가 유튜브 '엘르 코리아' 채널에 출연해 'Attention'...,0.520487
4,"뉴진스 민지, 배우 분위기 비결은 “톤온톤 사복 패션?”",20일 매거진 엘르 코리아 유튜브 채널에는 ‘강해린 이상하다’ ESTJ 민지의 멤버...,0.506953


In [18]:
summary = dict(zip(df['Title'].values, df['Content'].values))
summary

{"뉴진스, 데뷔음반도 밀리언셀러…'OMG' 이어 두 번째": "뉴진스는 데뷔음반도 밀리언셀러...'OMG' 이어 두 번째 등록을 마쳤으며 08:19:27에 09번째 등록을 마쳤다.",
 '하이브 작년 매출 1.7조 역대 최대…"지민 3월 솔로 앨범"(종합2보)': '그룹 방탄소년단(BTS)의 소속사 하이브가 글로벌 팬덤 확장과 신인의 성공적인 데뷔 등으로 지난해 창사 이래 최대 매출을 올렸으며 21일 오후 국내·외 증권사 애널리스트와 주요 기관 투자자를 대상으로 기업 설명회를 개최했다.',
 "뉴진스, 미국 누적 앨범 판매량 10만장 돌파..민희진도 '축하'": '스타뉴스 가요 담당 윤상근 기자입니다.',
 '"그게 사실은…" 뉴진스 민지, 레전드 \'비눗방울\' 직캠 비하인드 공개': "그룹 뉴진스 멤버 민지가 유튜브 '엘르 코리아' 채널에 출연해 'Attention' 무대에서 특수효과로 뿌려진 비눗방울을 톡 터트리며 센스있게 안무를 소화한 비하인드를 공개했다.",
 '뉴진스 민지, 배우 분위기 비결은 “톤온톤 사복 패션?”': '20일 매거진 엘르 코리아 유튜브 채널에는 ‘강해린 이상하다’ ESTJ 민지의 멤버 분석?! 뉴진스 민지의 #ASKMEANYTHING’이라는 제목의 영상이 올라왔다.'}

In [19]:
len(summary)

5

In [24]:
summary

{"뉴진스, 데뷔음반도 밀리언셀러…'OMG' 이어 두 번째": "뉴진스는 데뷔음반도 밀리언셀러...'OMG' 이어 두 번째 등록을 마쳤으며 08:19:27에 09번째 등록을 마쳤다.",
 '하이브 작년 매출 1.7조 역대 최대…"지민 3월 솔로 앨범"(종합2보)': '그룹 방탄소년단(BTS)의 소속사 하이브가 글로벌 팬덤 확장과 신인의 성공적인 데뷔 등으로 지난해 창사 이래 최대 매출을 올렸으며 21일 오후 국내·외 증권사 애널리스트와 주요 기관 투자자를 대상으로 기업 설명회를 개최했다.',
 "뉴진스, 미국 누적 앨범 판매량 10만장 돌파..민희진도 '축하'": '스타뉴스 가요 담당 윤상근 기자입니다.',
 '"그게 사실은…" 뉴진스 민지, 레전드 \'비눗방울\' 직캠 비하인드 공개': "그룹 뉴진스 멤버 민지가 유튜브 '엘르 코리아' 채널에 출연해 'Attention' 무대에서 특수효과로 뿌려진 비눗방울을 톡 터트리며 센스있게 안무를 소화한 비하인드를 공개했다.",
 '뉴진스 민지, 배우 분위기 비결은 “톤온톤 사복 패션?”': '20일 매거진 엘르 코리아 유튜브 채널에는 ‘강해린 이상하다’ ESTJ 민지의 멤버 분석?! 뉴진스 민지의 #ASKMEANYTHING’이라는 제목의 영상이 올라왔다.'}

In [25]:
import json

In [30]:
json_val = json.dumps(summary)
type(json.loads(json_val))

dict